# SQL tutorial using a database of drinks and bars

## Things we will be doing:
- Creating a PostgreSQL database using the `psql` shell tool
- Importing existing data to fill the database (the `drinks.sql` file in this repository)
- Using the `psycopg2` Python package to interact with the database (we could have alternatively used a dedicated SQL IDE like SQL Workbench)
- Quering the database to explore SQL concepts such as:
    - Filtering, ordering, limiting, etc.
    - Joining tables
    - Grouping records
    - Aggregate functions
    
## Requirements:
- Install PostgreSQL (and potentially give the `postgres` user a password)
- Install the `psycopg2` Python package in your anaconda virtual environment of choice. **Note**: you will need to install other dependencies beforehand. Look up which ones for your operating system.
    - Using conda: `conda install -c anaconda psycopg2`
    - Using pip: `pip install psycopg2`

# Creating the database using `psql`

1. Start up your terminal.
2. Switch to the "postgres" user that was created automatically when installing PostgreSQL (find the equivalent command for Windows). \
`sudo -u postgres -i`
3. Start the `psql` shell tool. \
`psql`
4. List the PostgreSQL databases that exist on your computer (press "q" when you want to exit the list view). \
`\l`
5. Create a new empty database called "drinks". \
`CREATE DATABASE drinks;`
4. Confirm that the "drinks" database was created (press "q" when you want to exit the list view). \
`\l`
5. We can now leave the `psql` tool. \
`exit`
5. And also log out of the "postgres" user. \
`exit`

# Create a Python connection to the database and load in some data from the `drinks.sql` file

In [3]:
import psycopg2
import pandas as pd

# Create a connection to the drinks database we just made (substitute the password you set for the "postgres" user)
con = psycopg2.connect(database='drinks', user='lhl_student', password='lhl_password',
                       host='127.0.0.1', port='5432')
cur = con.cursor()

cur.execute(open('../../migrations/drinks.sql', 'r').read())
con.commit()

In [4]:
# Using triple quotations will ensure that everything inside the string
# is read as a character (i.e. no need to use escape characters)
query = """
SELECT * 
FROM drinks
LIMIT 5;
"""

# You first execute the query, then get it's result.
# Note: if you try to chain .execute().fetchall() in the same statement,
# the database my not have time to execute the query in time, which will
# result in you getting an error.
cur.execute(query)
response = cur.fetchall()

# Let's look at the format of the response we get back
print(f'This is the raw response we get back:\n{response}\n')

# You can alternatively use pandas to get a nicely formatted DataFrame
pandas_response = pd.read_sql_query(query, con)
print('This is the nicely structured pandas response:\n')
pandas_response
con.close()

This is the raw response we get back:
[('drink 1', 'cocktail'), ('drink 2', 'wine'), ('drink 3', 'rum'), ('drink 4', 'cocktail'), ('drink 5', 'cocktail')]

This is the nicely structured pandas response:



/opt/homebrew/Caskroom/miniforge/base/envs/base_env/lib/python3.8/site-packages/pandas/io/sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [5]:
# Before doing anything else, let's create a function out
# of the things we're doing above
def execute_query(query_string, return_pandas=True, limit=' LIMIT 20;'):
    con = psycopg2.connect(database='drinks', user='lhl_student', password='lhl_password',
                       host='127.0.0.1', port='5432')
    cur = con.cursor()
    if limit:
        query_string+=limit
    if return_pandas:
        response = pd.read_sql_query(query_string, con)
    else:
        cur.execute(query_string)
        response = cur.fetchall()
    con.close()
    return response


# Let's try out our function to make sure it does
# the same as what we have above

query = """
SELECT * 
FROM drinks
"""
execute_query(query, limit=None)

/opt/homebrew/Caskroom/miniforge/base/envs/base_env/lib/python3.8/site-packages/pandas/io/sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,drink_id,type
0,drink 1,cocktail
1,drink 2,wine
2,drink 3,rum
3,drink 4,cocktail
4,drink 5,cocktail
5,drink 6,wine
6,drink 7,cocktail
7,drink 8,whisky
8,drink 9,rum
9,drink 10,wine


# Some SQL references before we get started
![](../../assets/sql-cheat-sheet.png)

## Order of operations
![](../../assets/order_of_operations.png)

# Time to start writing some queries

## Problem 1
Get the bar name and average price of drinks at each bar.

In [9]:
execute_query("""
SELECT name, average
""")

DatabaseError: Execution failed on sql '
SELECT name, average
 LIMIT 20;': column "name" does not exist
LINE 2: SELECT name, average
               ^


## Problem 2
Get the bars with the top 5 average prices.

In [ ]:
execute_query("""

""", )

## Problem 3
Get the bar with the cheapest drink, along with the drink type and price.

In [ ]:
execute_query("""

""", )

## Problem 4
Get the number of beers sold by each bar in descending order (number of beers sold, not the number of beers on their menu).

In [ ]:
execute_query("""
SELECT bar, COUNT(*) as beersSold
FROM orders
LEFT JOIN drinks ON orders.drink_id = drinks.drink_id
WHERE drinks.type = 'beer'
group by 1
order by 2 desc
""")

## Problem 5
For each person, find the bar they visit, and the type(s) and price(s) of the drink(s) they drink during those visits.

In [ ]:
execute_query("""
select 
distinct 
o.person, o.bar, d.type, mi.price
FROM orders o
LEFT JOIN menu_items mi on o.drink_id = mi.drink_id and o.bar = mi.bar
LEFT JOIN drinks d on d.drink_id = o.drink_id
""")

## Bonus: Problem 6
Your restraunt chain is loosing business, how can we find out what the last bar a person visited, can we attribute this bar to being a root cause to why our customers are leaving ?

In [ ]:
execute_query("""
select * from (
select person, date, bar, row_number() over (partition by person order by date desc) as vist_number
from orders
) as t
where t.vist_number = 1
""", limit=None)

In [ ]:
execute_query("""
select max(date), min(date)
from orders
""")

## Drinks Database Schema Diagram
<img src="../../assets/drinks_schema_diagram.png" width="750"/>

# Below are my answers

## Answer 1

In [ ]:
execute_query("""
SELECT bar, AVG()
""")

## Answer 2

In [ ]:
execute_query("""
SELECT bar, AVG(price) AS avg_price
FROM menu_items
GROUP BY bar
ORDER BY avg_price DESC
""", limit = 'LIMIT 5')

## Answer 3

In [ ]:
execute_query("""
SELECT bar, drink_id, price
FROM menu_items
ORDER BY price ASC
""", limit='LIMIT 1;')

## Answer 4

In [ ]:
execute_query("""
SELECT orders.bar, SUM(orders.quantity) as beers_sold 
FROM orders
JOIN drinks ON drinks.drink_id = orders.drink_id
WHERE drinks.type LIKE '%beer%'
GROUP BY orders.bar
ORDER BY beers_sold DESC
""")

## Answer 5

In [ ]:
execute_query("""
SELECT o.person, o.bar, d.type, d.drink_id, h.price
FROM orders AS o
JOIN menu_items AS h ON (o.drink_id = h.drink_id AND o.bar = h.bar) 
JOIN drinks AS d ON o.drink_id = d.drink_id
GROUP BY o.person, o.bar, d.type, d.drink_id, h.price
""")

# Remember to close your database connection when you're done.
**Note**: If you run this cell then try to run queries, it won't work.
You will have to re-run the cell that initialized the connection.

In [ ]:
con.close()